In [3]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [4]:
# Set Hyperparameters
LR = 0.0002
GAMMA = 0.98

In [8]:
# Set Model
class TD_AC(nn.Module):
  def __init__(self):
    super(TD_AC,self).__init__()
    self.data = []

    self.fc1 = nn.Linear(4,256)
    self.fc_pi = nn.Linear(256,2)
    self.fc_v = nn.Linear(256,1)
    self.optimizer = optim.Adam(self.parameters(),lr = LR)

  def pi(self,x,softmax_dim=0):
    x = F.relu(self.fc1(x))
    prob = F.softmax(self.fc_pi(x),dim=softmax_dim)
    return prob

  def v(self,x):
    x = F.relu(self.fc_1(x))
    v = self.fc_v(x)
    return v

  def put_data(self,transition):
    self.data.append(transition)

  def make_batch(self):
    s_lst, a_lst, r_lst, s_prime_lst, prob_lst, done_lst =[],[],[],[],[],[]

    for transition in self.data:

      s,a,r,s_prime,prob,done = transition
      s_lst.append(s)
      a_lst.append([a])
      r_lst.append([r/100.0])
      s_prime_lst.append(s_prime)
      prob_lst.append([prob])
      done_mask = 0.0 if done else 1.0
      done_lst.append([done_mask])

      s,a,r,s_prime,prob,done = torch.tensor(s_lst,dtype=torch.float),torch.tensor(a_lst),torch.tensor(r_lst),torch.tensor(s_prime_lst,dtype=torch.float),torch.tensor(prob_lst),torch.tensor(done_lst,dtype=torch.float)
      self.data = []
      return s,a,r,s_prime,prob,done

    # Set train net
  def train_net(self):
    s,a,r,s_prime,prob,done = self.make_batch
    td_target = r + GAMMA * self.v(s_prime)
    td_error = td_target - self.v

    pi = self.pi(s,softmax_dim=1)
    pi_a = pi.gather(1,a)

    loss = -torch.log(pi_a) * td_error.detach() + F.smooth_l1_loss(td_target.detach(),self.v(s))

    self.optimizer.zero_grad()
    loss.mean().backward()
    self.optimizer.step()

In [13]:
def main():
  env = gym.make('CartPole-v1')

  # Initalize
  model = TD_AC()
  print_interval = 20
  score = 0.0

  for n_epi in range(10000):
    done = False
    s= env.reset()
    while not done:
      for t in range(10):
        prob = model.pi(torch.from_numpy(s).float())
        a = Categorical(prob).sample().item()
        s_prime, r, done, truncated, info = env.step(a)
        model.put_data((s,a,r,s_prime,prob[a],done))

        s = s_prime
        score += r

        if done:
          break

        model.train_net()

      if n_epi%print_interval==0 and n_epi!=0:
          print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
          score = 0.0
    env.close()


In [14]:
main()

/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


AttributeError: module 'numpy' has no attribute 'bool8'